In [1]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import selenium
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import time
from selenium.common.exceptions import NoSuchElementException


### 1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")



C:\Users\geopt\AppData\Local\Temp\ipykernel_24152\2798857193.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")


In [3]:
#open amazon
driver.get("https://www.amazon.in")

In [4]:
item = input("Enter Item to search: ")

Enter Item to search: guitar


In [5]:
#enter item
search=driver.find_element(By.XPATH,"/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input")
# search = driver.find_element(By.ID,"twotabsearchtextbox")
search.send_keys(item)

In [6]:
#submit
btn=driver.find_element(By.ID,"nav-search-submit-button")
btn.click()


#### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [7]:
#collecting all the product page urls
urls = []
for i in range(0,3):
    page_urls=driver.find_elements(By.XPATH,"//a[@class='a-link-normal s-no-outline']")
    for i in page_urls:
        urls.append(i.get_attribute('href'))
        
    #next page button
    nxt_btn=driver.find_element(By.XPATH,"//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']") 
    url=nxt_btn.get_attribute('href')
    driver.get(url)
    time.sleep(2)

In [11]:
#init variables
brand_name= []
product_name = []
price = []
return_exchange = []
expected_delivery = []
availability = []


for i in urls:
    driver.get(i)
    time.sleep(2)
    
    #Scraping brand name
    try:
        brand=driver.find_element(By.XPATH,"//tr[@class='a-spacing-small po-brand']/td[2]")
        brand_name.append(brand.text)
    except NoSuchElementException as e:
        brand_name.append("-")
    
    #Scraping product name
    try:
        prod=driver.find_element(By.XPATH,"//span[@id='productTitle']")
        product_name.append(prod.text)
    except NoSuchElementException as e:
        product_name.append("-")
        
        
    #Scraping Price
    try:
        pri=driver.find_element(By.XPATH,'//span[@class="a-price aok-align-center reinventPricePriceToPayMargin priceToPay"]')  
        price.append(pri.text)
    except NoSuchElementException as e:
        price.append("-")
        
     
    #Scraping Return/Exchange
    try:
        ret=driver.find_element(By.XPATH,"//div[@data-name='RETURNS_POLICY']/span/div[2]/a")
        return_exchange.append(ret.text)
    except NoSuchElementException as e:
        return_exchange.append("-")
        
     
    #Scraping  Expected Delivery
    try:
        deliv=driver.find_element(By.XPATH,"//div[@id='mir-layout-DELIVERY_BLOCK']/div/span/span")
        expected_delivery.append(deliv.text)
    except NoSuchElementException as e:
        expected_delivery.append("-")
        
        
    #Scraping Availability
    try:
        avai=driver.find_element(By.XPATH,"//div[@id='availability']/span")
        availability.append(avai.text)
    except NoSuchElementException as e:
        availability.append("-")

# create dataframe
Guitar=pd.DataFrame({})
Guitar['Brand'] = brand_name
Guitar['Name'] = product_name
Guitar['Price'] = price
Guitar['Return'] = return_exchange
Guitar['Expected_Delivery'] = expected_delivery
Guitar['Availability'] = availability
Guitar['URL'] = urls

In [16]:
Guitar

,Brand,Name,Price,Return,Expected_Delivery,Availability,URL
0,Kadence,Kadence Slowhand Premium Jumbo Semi Acoustic G...,"₹10,499\n00",7 days Replacement,"Wednesday, 2 November",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,VAULT,Vault EA40 41 inch Premium Solid Spruce-Top Cu...,"₹7,290\n00",7 days Replacement,"Tuesday, 1 November",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Cort,Cort AD810 Dreadnought Acoustic Guitar with Gi...,"₹10,900\n00",7 days Replacement,"Tuesday, 1 November",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Juârez,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","₹2,399\n00",7 days Replacement,"Thursday, 3 November",In stock.,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
4,Intern,"Intern INT-38C Right hand Acoustic Guitar Kit,...","₹2,189\n00",7 days Replacement,"Thursday, 3 November",In stock.,https://www.amazon.in/Intern-INT-38C-Acoustic-...
...,...,...,...,...,...,...,...
148,JUAREZ,JUAREZ JRZ41C 41 Inch ACOUSTIC GUITAR WITH ACC...,"₹3,534\n49",7 days Replacement,"Monday, 31 October",In stock.,https://www.amazon.in/JUAREZ-JRZ41C-ACOUSTIC-G...
149,JN Guitars,JN Guitars Bessie Series 6 String Acoustic Gui...,-,Returns Policy,-,,https://www.amazon.in/JN-Guitars-Acoustic-MINI...
150,YAMAHA,Yamaha PACIFICA012 Electric Guitar - Black,"₹15,334\n00",7 days Replacement,"Tuesday, 1 November",In stock.,https://www.amazon.in/Yamaha-Pacifica-PAC012-E...
151,YAMAHA,"Yamaha F370 Acoustic Guitar, Natural","₹13,989\n00",7 days Returnable,November 5 - 8,Only 1 left in stock.,https://www.amazon.in/Yamaha-F370-Acoustic-Gui...


In [17]:
Guitar.to_csv('guitar_list.csv') # Saving as .csv file

In [20]:
driver.quit()

#### 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [23]:
driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")

C:\Users\geopt\AppData\Local\Temp\ipykernel_24152\3864639957.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")


In [29]:
# search list
items = ['fruits','cars','Machine Learning','Guitar','Cakes'] 

# google url
url = "https://images.google.com/"

#init result lists 
urls = []    
data = []


for item in items:
    driver.get(url)  
    time.sleep(3)
    search_bar = driver.find_element("name", "q") # search bar
    
    search_bar.send_keys(str(item)) #sending key word for search item
    
    search_button =driver.find_element(By.XPATH,"//button[@class='Tg7LZd']").click() #Clicking on search button
    
    # scrolling the web page to get more images
    for _ in range(50):
        driver.execute_script("window.scrollBy(0,100)")
     
    # to get image url
        imgs = driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:10]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 50:
        break
    print("Downloading {0} of 50 images" .format(i+1))
    response = requests.get(urls[i])
    
    # To save downloaded pics in local folder
    file = open(r"E:\Downloads-Temp\fliprobo\google_images"+str(i)+".jpg", "wb")

    file.write(response.content)

In [30]:
driver.quit()

#### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV

In [31]:
driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")

C:\Users\geopt\AppData\Local\Temp\ipykernel_24152\3864639957.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")


In [33]:
#open flipkart
driver.get("https://www.flipkart.com")

In [36]:
# close login pop up
driver.find_element(By.XPATH,"//button[@class='_2KpZ6l _2doB4z']").click()

In [38]:
# to enter the item name which user wants to search
item = input(" Enter the phone name : ")

#giving input key word to search bar
serch_bar = driver.find_element(By.XPATH,"//div[@class='_3OO5Xc']//input")
serch_bar.send_keys(item)

# to click search button
srch_btn = driver.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
srch_btn.click()
time.sleep(5)

 Enter the phone name : oneplus nord


In [46]:
# To get  urls of phones on 1st page
urls = []
url_tag = driver.find_elements(By.CLASS_NAME,'_1fQZEK')
for url in url_tag:
    urls.append(url.get_attribute("href"))


In [49]:
# entry count
len(urls)

24

In [58]:
#init list keys
Smartphones = {}
Smartphones["Brand"] = []
Smartphones["Phone name"] = []
Smartphones["Colour"] = []
Smartphones["RAM"] = []
Smartphones["Storage(ROM)"] = []
Smartphones["Primary Camera"] = []
Smartphones["Secondary Camera"] = []
Smartphones["Display Size"] = []
Smartphones["Battery Capacity"] = []
Smartphones["Price"] = []
Smartphones["URL"] = []

# Scraping data from each product
for url in urls:
    driver.get(url)                                                        
    print("Scraping URL = ", url)
    Smartphones['URL'].append(url)                                                          
    time.sleep(2)
    
    
    #Clicking on read more button
    try:
        read_more = driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
    
    
    #Scraping brand name of phone data
    try:
        brand_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')      
        Smartphones["Brand"].append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Smartphones['Brand'].append('-')
    
    
    #Scraping phone name data
    try:
        name_tags = driver.find_element(By.XPATH,'//span[@class="B_NuCI"]')     
        Smartphones['Phone name'].append(name_tags.text)
    except NoSuchElementException:
        Smartphones['Phone name'].append('-')
    
    
    #Scraping phone color data
    try:
        color_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Smartphones['Colour'].append(color_tags.text)
    except NoSuchElementException:
        Smartphones['Colour'].append('-')
     
    
    #Scraping RAM data
    try:
        ram_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        Smartphones['RAM'].append(ram_tags.text)
    except NoSuchElementException:
        Smartphones['RAM'].append('-')
    
    
    #Scraping ROM data
    try:
        rom_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        Smartphones['Storage(ROM)'].append(rom_tags.text)
    except NoSuchElementException:
        Smartphones['Storage(ROM)'].append('-')
        
        
    #Scraping Primary camera data
    try:                                                                                    
        pri_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Smartphones['Primary Camera'].append(pri_tags.text)
    except NoSuchElementException:
        Smartphones['Primary Camera'].append('-')
        
        
    #Scraping secondary camera data
    try:                                                                                    
        sec_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Smartphones['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        Smartphones['Secondary Camera'].append('-')
        
        
    #Scraping Display size data 
    try:
        disp_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Smartphones['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        Smartphones['Display Size'].append('-')
    
    
    #Scraping battery capacity data
    try:
        if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Smartphones['Battery Capacity'].append(bat_capa.text)
    except NoSuchElementException:
        Smartphones['Battery Capacity'].append('-')
        
        
        
    #Scraping Price data
    try:
        price_tags = driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')      
        Smartphones['Price'].append(price_tags.text)
    except NoSuchElementException:
        Smartphones['Price'].append('-')

Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-black-dusk-128-gb/p/itm853d95cf27843?pid=MOBGG6SGT5TVGSG6&lid=LSTMOBGG6SGT5TVGSG6EXWFNK&marketplace=FLIPKART&q=oneplus+nord&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=2ab07df3-aff0-49f3-a5bf-03ba5f17d79d.MOBGG6SGT5TVGSG6.SEARCH&ppt=hp&ppn=homepage&ssid=xfifrg2xb40000001666984073827&qH=e8b736bf4fa84421
Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-blue-tide-128-gb/p/itm853d95cf27843?pid=MOBGG6SGBYXGNYMH&lid=LSTMOBGG6SGBYXGNYMHNDQ9BS&marketplace=FLIPKART&q=oneplus+nord&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=2ab07df3-aff0-49f3-a5bf-03ba5f17d79d.MOBGG6SGBYXGNYMH.SEARCH&ppt=hp&ppn=homepage&ssid=xfifrg2xb40000001666984073827&qH=e8b736bf4fa84421
Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-black-dusk-128-gb/p/itm7acae55b999e6?pid=MOBGDWF8GJTFAXVH&lid=LSTMOBGDWF8GJTFAXVHSZ1LDM&marketplace=FLIPKART&q=oneplus+nord&store=

In [59]:
#Data Frame the data
df = pd.DataFrame.from_dict(Smartphones)
df

,Brand,Phone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,URL
0,Nord,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)...",Black Dusk,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹19,990",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,Nord,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB) ...",Blue Tide,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹19,998",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,Nord,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)...",Black Dusk,6 GB,128 GB,Primary Camera,-,16.74 cm (6.59 inch),-,"₹19,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
3,Nord,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)...",Black Dusk,8 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹21,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
4,Nord,"OnePlus Nord 2T 5G (Gray Shadow, 256 GB) (12 ...",Gray Shadow,12 GB,256 GB,Primary Camera,-,17.02 cm (6.7 inch),-,"₹33,999",https://www.flipkart.com/oneplus-nord-2t-5g-gr...
5,Nord,"OnePlus Nord CE2 Lite 5G (Blue Tide, 128 GB) ...",Blue Tide,8 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹21,900",https://www.flipkart.com/oneplus-nord-ce2-lite...
6,Nord,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB) ...",Blue Tide,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹19,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
7,Nord,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)...",Black Dusk,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹19,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
8,Nord,"OnePlus Nord 2T 5G (Jade Fog, 256 GB) (12 GB ...",Jade Fog,12 GB,256 GB,Primary & Secondary Camera,-,17.02 cm (6.7 inch),-,"₹33,990",https://www.flipkart.com/oneplus-nord-2t-5g-ja...
9,Nord,"OnePlus Nord 2T 5G (Jade Fog, 256 GB) (12 GB ...",Jade Fog,12 GB,256 GB,Primary & Secondary Camera,-,17.02 cm (6.7 inch),-,"₹33,990",https://www.flipkart.com/oneplus-nord-2t-5g-ja...


In [61]:
df.to_csv('smart_phones.csv') # Saving as .csv file

In [62]:
driver.quit()

#### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps

In [63]:
driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")

C:\Users\geopt\AppData\Local\Temp\ipykernel_24152\3864639957.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")


In [64]:
# opening google maps
driver.get('https://www.google.co.in/maps')

In [67]:
# importing regex
import re

#Sending keyword for seach bar and search button
city = input('Enter City name that has to be searched : ')
search_bar = driver.find_element(By.ID,"searchboxinput")                       
search_bar.clear()                                                             
time.sleep(2)
search_bar.send_keys(city)                                                     
search_btn = driver.find_element(By.ID,"searchbox-searchbutton")              
search_btn.click()                                                             
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City name that has to be searched : calicut
URL Extracted:  https://www.google.co.in/maps/place/Kozhikode,+Kerala/@11.255802,75.6707217,11z/data=!3m1!4b1!4m5!3m4!1s0x3ba65938563d4747:0x32150641ca32ecab!8m2!3d11.2587531!4d75.78041
Latitude = 11.255802, Longitude = 75.6707217


In [68]:
driver.quit()

#### 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [69]:
driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")

C:\Users\geopt\AppData\Local\Temp\ipykernel_24152\3864639957.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")


In [70]:
# opening trak
driver.get('https://trak.in')

In [72]:
fund_button = driver.find_element(By.XPATH,'//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(fund_button)

In [75]:
# init
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []


In [76]:
for i in range(54,57):
    # Date
    dt = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


In [77]:
driver.quit()

#### 7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [78]:
driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")

C:\Users\geopt\AppData\Local\Temp\ipykernel_24152\3864639957.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")


In [79]:
# open digit.in
driver.get('https://digit.in')

In [80]:
#search for best laptop
best_gam_lap = driver.find_element(By.XPATH,"//div[@class='listing_container']//ul//li[9]").click()


In [85]:
#Creating empty lists
lap_name = []
ope_sys = []
display = []
processor = []
memory = []
weight = []
dimensions = []
price = []
# to Scrape laptop names
name_tags = driver.find_elements(By.XPATH,"//table[@id='summtable']//tr//td[1]")
for name in name_tags:
    lap_name.append(name.text)
    
    
# to Scrape the operating system
try:
    os_tags = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']//tr[3]//td[3]")
    for os in os_tags:
        ope_sys.append(os.text)
except NoSuchElementException:
    pass


# to Scrape display
try:
    disp_tags = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']//tr[4]//td[3]")
    for disp in disp_tags:
        display.append(disp.text)
except NoSuchElementException:
    pass


#to Scrape Processor
try:
    pro_tags = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']//tr[5]//td[3]")
    for pro in pro_tags:
        processor.append(pro.text)
except NoSuchElementException:
    pass


# to Scrape memory
try:
    memo_tags = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']//tr[6]//td[3]")
    for memo in memo_tags:
        memory.append(memo.text)
except NoSuchElementException:
    pass


# to Scrape weight
try:
    wgt_tags = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']//tr[7]//td[3]")
    for wgt in wgt_tags:
        weight.append(wgt.text)
except NoSuchElementException:
    pass


#to Scrape dimensions
try:
    dim_tags = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']//tr[8]//td[3]")
    for dim in dim_tags:
        dimensions.append(dim.text)
except NoSuchElementException:
    pass




#to Scrape price
try:
    pri_tags = driver.find_elements(By.XPATH,"//td[@class='smprice']")
    for pri in pri_tags:
        price.append(pri.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

In [86]:
# create dataframe
Gaming_Laptop=pd.DataFrame({})
Gaming_Laptop['Laptop Name'] = lap_name
Gaming_Laptop['Operating system'] = ope_sys
Gaming_Laptop['Display'] = display
Gaming_Laptop['Processor'] = processor
Gaming_Laptop['Memory'] = memory
Gaming_Laptop['Weight'] = weight
Gaming_Laptop['Dimensions'] = dimensions
Gaming_Laptop['Price'] = price

In [84]:
Gaming_Laptop

,Laptop Name,Operating system,Display,Processor,Memory,Weight,Dimensions,Price
0,MSI Titan GT77-12UHS,12th Gen Intel Core i9-12900HX 16 core processor,"17.3″ (3840 x 2160) screen, 120 Hz refresh rate",Windows 11 Home,64 GB DDR5 RAM & 2 TB SSD,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,397 x 330 x 23 mm dimension & 3.3 kg weight,N/A
1,Alienware x17 R2,12th Gen Intel Core i9-12900H 14 core processo...,"17.3″ (1920 x 1080) screen, 360 Hz refresh rate",Windows 11 Home,32 GB DDR5 RAM & 1 TB SSD,16 GB DDR6 NVIDIA GEFORCE RTX 3080 Ti Graphics...,299 x 359 x 21 mm dimension & 2.96 kg weight,"Rs 389,990"
2,Acer Predator Triton 500 SE PT516-52s,12th Gen Intel Core i7-12700H 14 core processo...,"16″ (2560 x 1600) screen, 240 Hz refresh rate",Windows 11 Home,32 GB DDR5 RAM & 2 TB SSD,8 GB DDR6 NVIDIA GeForce RTXTM 3070 Ti Graphic...,358 x 262 x 19.9 mm dimension & 2.4 kg weight,"Rs 300,000"
3,Omen by HP (16-B1371TX),12th Gen Intel Core i7-12700H 14 core processo...,16.1″ (2560 x 1440) screen,Windows 11 Home,8 GB DDR5 RAM & 1 TB SSD,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,369 x 248 x 23 mm dimension & 2.32 kg weight,"Rs 176,290"
4,Acer Predator Helios 300 AN515-45 (NH.QBRSI.0,AMD Ryzen 9-5900HX 8 core processor,"15.6″ (2560 x 1440) screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,363 x 255 x 23.9 mm dimension & 2.4 kg weight,"Rs 172,999"
5,MSI Delta 15 (A5EFK-083IN),AMD 5th Gen Ryzen 9-5900HX 8 core processor wi...,"15.6″ (1920 x 1080) screen, 240Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,10 GB DDR6 AMD Radeon RX 6700M Graphics card,357 x 247 x 19 mm dimension & 1.9 kg weight,"Rs 188,990"
6,Omen by HP (16-c0141AX),AMD Ryzen™ 9 5900HX 8 core processor with 3.3 ...,16.1″ (2560 x 1440) screen,Windows 11 Home,16 GB DDR4 RAM & 1 TB NVMe,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight,"Rs 124,990"
7,Lenovo Legion 5i Pro (82RF00MGIN),12th Gen Intel Core i7-12700H 14 core processo...,"16″ (2560 x 1600) screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR5 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,360 x 264 x 20 mm dimension & 2.49 kg weight,"Rs 168,500"
8,Alienware m15 R5 Ryzen Edition icc-c780001win,AMD Ryzen R7-5800H 8 core processor with 4.40 ...,"15.6″ (1920 x 1080) screen, 165Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...,"Rs 144,990"
9,Lenovo Slim 7 Gen 6 (82K8002JIN),Lenovo Ryzen 7-5800H processor with 3.2 GHz cl...,"15.6″ screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,356 x 252 x 16 mm dimension & 1.9 kg weight,"Rs 131,990"


In [87]:
driver.quit()

#### 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”

In [101]:
driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")

C:\Users\geopt\AppData\Local\Temp\ipykernel_24152\3864639957.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")


In [106]:
# forbes.com
driver.get('https://www.forbes.com')

In [107]:
#clicking the explore button
button = driver.find_element(By.XPATH,"/html/body/div[1]/header/nav/div[1]/div/div/div[1]")
button.click()
time.sleep(1)

#select billioners 
billioners = driver.find_element(By.XPATH,"/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[1]/a")
billioners.click()
time.sleep(2)

#select world billioners 
world_billioners= driver.find_element(By.XPATH,"/html/body/div[1]/main/div/section/section[1]/div/div/div[1]/div/div[1]/div[1]/div[2]/a")
world_billioners.click()
time.sleep(1)

In [109]:
#init

Rank = []
Person_Name = []
total_net_worth = []
Age = []
citizenship = []
Source = []
industry = []



while(True):
    #to Scrape rank
    rank_tags= driver.find_elements(By.XPATH,"//div[@class='rank']")
    for rank in rank_tags:
        Rank.append(rank.text)
    time.sleep(1)
    
    
    #to Scrape names
    name_tags= driver.find_elements(By.XPATH,"//div[@class='personName']//div")
    for name in name_tags:
        Person_Name.append(name.text)
    time.sleep(1)
    
    
    #to Scrape age
    age_tags= driver.find_elements(By.XPATH,"//div[@class='age']//div")
    for age in age_tags:
        Age.append(age.text)   
    time.sleep(1)
    
    
    #to Scrape citizenship
    cit_tags= driver.find_elements(By.XPATH,"//div[@class='countryOfCitizenship']")
    for cit in cit_tags:
        citizenship.append(cit.text)
    time.sleep(1)
    
    
    #to Scrape source of income
    sour_tags= driver.find_elements(By.XPATH,"//div[@class='source']")
    for sour in sour_tags:
        Source.append(sour.text)    
    time.sleep(1)
    
    
    #to Scrape Industry
    ind_tags= driver.find_elements(By.XPATH,"//div[@class='category']//div")
    for ind in ind_tags:
        industry.append(ind.text)
        
        
    #to scrape net_worth of billionaire 
    net_tags= driver.find_elements(By.XPATH,"//div[@class='netWorth']//div[1]")
    for net in net_tags:
        total_net_worth.append(net.text)
    time.sleep(1)
    
    
    #to Click on next button
    try:
        next_button = driver.find_element(By.XPATH,"//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break
        
#to Scrape net worth
Net_Worth = []
for i in range(0,len(total_net_worth),2):
    Net_Worth.append(total_net_worth[i])

In [110]:
#dataframe
Billionaires=pd.DataFrame({})
Billionaires['Rank'] = Rank
Billionaires['Names'] = Person_Name
Billionaires['Net Worth'] = Net_Worth
Billionaires['Age'] = Age
Billionaires['Citizenship'] = citizenship
Billionaires['Source'] = Source
Billionaires['Industry'] = industry
Billionaires

,Rank,Names,Net Worth,Age,Citizenship,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
2663,2578.,Zhang Yuqiang,$1 B,66,China,Fiberglass,Manufacturing
2664,2578.,Zhou Ruxin,$1 B,59,China,Navigation,Technology
2665,2578.,Wen Zhou & family,$1 B,57,China,chemicals,Manufacturing
2666,2578.,Zhou Yifeng & family,$1 B,43,China,liquefied petroleum gas,Energy


In [176]:
driver.quit()

#### 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [177]:
driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")

C:\Users\geopt\AppData\Local\Temp\ipykernel_24152\3864639957.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")


In [181]:
# opening google maps
driver.get('https://www.youtube.com/')

In [182]:
# input the keywords that user wants to search
user_input = input('Enter the video : ')

#finding element for search bar
search_bar = driver.find_element(By.XPATH,'//input[@id="search"]')
search_bar.send_keys(user_input)
time.sleep(1)

#clicking on search button
search_btn = driver.find_element(By.ID,"search-icon-legacy")  
search_btn.click()
time.sleep(1)

Enter the video : cricket


In [183]:
#clicking on first video
link_click = driver.find_element(By.XPATH,"//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
link_click.click()

# scroll down to generate more Comments
for _ in range(3000):
    driver.execute_script("window.scrollBy(0,3000)")

In [184]:
# empty lists
comments = []

# scrape comments
cm_tags = driver.find_elements(By.ID,"content-text")
for cm in cm_tags:
    if cm.text is None:
        comments.append("-")
    else:
        comments.append(cm.text)
time.sleep(5)
comments[:500]


['Hats off to Zimababwe for the way they got Zimbabar and Zimwan out for single digit scores',
 "A lesson can be learn from Yesterday's match that overconfidence can ruin chances of victory  Waqt, Jazbaat sab kuch badal jaate hain ",
 "I don't know why Pakistanis are upset. Pakistan vs Zimbabwe is the El-Classico of Pakistan. Sometimes you win sometimes you lose.",
 'Such comments really pull up a team .Now India will surely win the Cup.Thanks Shoaib.',
 'Even in cricket, Pakistan survives only by mentioning India.',
 'Meaning of the flag🇵🇰\nBegging bowl \n One coin falling in it ',
 'Jab aadmi ki shakal achi na ho to usse baate achi karni chahiye… !!\n\nPakistani and Pakistan don’t need to worry about Team India…we have billion people to support our team.',
 'A Day without Humiliation is Day wasted\n                   -- Beggisthan ️ 🇵🇰',
 '1 thing which paxtan has mastered in the recent years is losing the cricket match in the last ball',
 'It is not the biggest upset of T20 world cu

In [185]:
len(comments)

374

In [186]:
# scrape time 
Time = []

tm_tags = driver.find_elements(By.XPATH,"//yt-formatted-string[@class='published-time-text style-scope ytd-comment-renderer']")
for tm in tm_tags:
    if tm.text is None:
        Time.append("-")
    else:
        Time.append(tm.text)

time.sleep(5)

['16 hours ago (edited)',
 '16 hours ago',
 '15 hours ago',
 '16 hours ago (edited)',
 '16 hours ago',
 '15 hours ago',
 '10 hours ago',
 '16 hours ago (edited)',
 '15 hours ago',
 '9 hours ago',
 '13 hours ago',
 '14 hours ago',
 '14 hours ago',
 '16 hours ago',
 '8 hours ago',
 '15 hours ago (edited)',
 '8 hours ago',
 '16 hours ago',
 '12 hours ago',
 '8 hours ago',
 '13 hours ago',
 '16 hours ago',
 '14 hours ago',
 '15 hours ago (edited)',
 '16 hours ago',
 '11 hours ago',
 '9 hours ago',
 '1 hour ago',
 '11 hours ago',
 '10 hours ago',
 '14 hours ago',
 '16 hours ago',
 '9 hours ago',
 '11 hours ago',
 '11 hours ago',
 '15 hours ago',
 '13 hours ago',
 '13 hours ago',
 '8 hours ago',
 '14 hours ago',
 '16 hours ago (edited)',
 '14 hours ago',
 '14 hours ago',
 '15 hours ago',
 '9 hours ago',
 '15 hours ago',
 '15 hours ago',
 '14 hours ago',
 '14 hours ago',
 '15 hours ago',
 '15 hours ago',
 '15 hours ago',
 '10 hours ago',
 '14 hours ago',
 '14 hours ago',
 '14 hours ago',
 '4 

In [192]:
Likes = []
# to scrape comment likes
like_tags = driver.find_elements(By.XPATH,"//span[@id='vote-count-middle']")
for like in like_tags:
    if like.text is None:
        Likes.append("--")
    else:
        Likes.append(like.text)
    
Likes[:500]  

['451',
 '270',
 '278',
 '110',
 '72',
 '187',
 '168',
 '188',
 '27',
 '21',
 '15',
 '92',
 '8',
 '58',
 '137',
 '4',
 '8',
 '12',
 '2',
 '39',
 '3',
 '5',
 '6',
 '',
 '8',
 '1',
 '34',
 '8',
 '1',
 '1',
 '7',
 '2',
 '',
 '1',
 '20',
 '',
 '1',
 '3',
 '',
 '',
 '',
 '',
 '10',
 '',
 '1',
 '1',
 '1',
 '3',
 '37',
 '',
 '1',
 '',
 '',
 '',
 '',
 '7',
 '',
 '',
 '',
 '',
 '4',
 '',
 '12',
 '1',
 '',
 '1',
 '',
 '2',
 '1',
 '2',
 '',
 '2',
 '',
 '6',
 '',
 '7',
 '',
 '',
 '1',
 '3',
 '27',
 '1',
 '',
 '',
 '1',
 '1',
 '1',
 '',
 '',
 '5',
 '',
 '3',
 '',
 '',
 '1',
 '',
 '2',
 '3',
 '3',
 '2',
 '1',
 '',
 '1',
 '',
 '',
 '',
 '',
 '',
 '1',
 '',
 '',
 '',
 '',
 '',
 '',
 '1',
 '',
 '',
 '1',
 '',
 '',
 '',
 '1',
 '',
 '',
 '3',
 '',
 '',
 '',
 '',
 '',
 '1',
 '',
 '1',
 '',
 '',
 '3',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '2',
 '',
 '',
 '1',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '5',
 '2',
 '',
 '1',
 '',
 '',
 '',
 '2',
 '',
 '',
 '',
 '',
 '',
 '',
 '

In [193]:
len(Likes)

374

In [205]:
#Creating dataframe
Youtube=pd.DataFrame({})
Youtube['Comments'] = comments[:500]
Youtube['Comment_time'] = Time[:500]
Youtube['Comment upvotes'] = Likes[:500]

#Printing dataframe
Youtube

,Comments,Comment_time,Comment upvotes
0,Hats off to Zimababwe for the way they got Zim...,16 hours ago (edited),451
1,A lesson can be learn from Yesterday's match t...,15 hours ago (edited),270
2,I don't know why Pakistanis are upset. Pakista...,16 hours ago,278
3,Such comments really pull up a team .Now India...,15 hours ago,110
4,"Even in cricket, Pakistan survives only by men...",15 hours ago,72
...,...,...,...
369,Average Bowler,9 hours ago,
370,Overated akhtar,15 hours ago,
371,,14 hours ago,
372,Coming from someone who himself defeated many ...,15 hours ago,


In [488]:
driver.quit()

## 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [587]:
driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")

C:\Users\geopt\AppData\Local\Temp\ipykernel_24152\3864639957.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(r"E:\Downloads-Temp\chromedriver_win32\chromedriver.exe")


In [588]:
# open hostelworld
driver.get('https://www.hostelworld.com')

In [589]:
#search London
# search = driver.find_element(By.ID,'location-text-input-field')
search = driver.find_element(By.CLASS_NAME,'location-text')
# search = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div/input')
search.send_keys(" ")
time.sleep(2)

In [591]:
search.send_keys("London")

In [592]:
#select london
click_search = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]')
# click_search = driver.find_element(By.ID,'search-button')
click_search.click()
time.sleep(2)

In [594]:
# click search button
search_btn = driver.find_element(By.ID,"search-button")
search_btn.click()

In [595]:


#collecting all the page urls
urls = []
hostel_name = []
distance = []
pvt_prices = []
dorms_price = []

# go to 1st page
try:
    nxt_btn=driver.find_element(By.XPATH,"//div[@class='pagination-item pagination-first']") 
    nxt_btn.click()
    time.sleep(2)
except:
    pass

for _ in range(50):
    driver.execute_script("window.scrollBy(0,8500)")
    
    
while(True):
    page_urls=driver.find_elements(By.XPATH,"//a[@class='view-button']")
    for i in page_urls:
        urls.append(i.get_attribute('href'))
        
        
    tags=driver.find_elements(By.XPATH,"//h2[@class='title title-6']")
    for i in tags:
        hostel_name.append(i.text)
        
    tags=driver.find_elements(By.XPATH,"//span[@class='description']")
    for i in tags:
        distance.append(i.text.replace('Hostel - ','').replace('Hotel - ','').replace('Bed and Breakfast - ','').replace(' from city centre',''))
        
    tags=driver.find_elements(By.XPATH,"//div[@class='prices-col']/a[1]/div[1]")
    for i in tags:
        pvt_prices.append(i.text)
        
    tags=driver.find_elements(By.XPATH,"//div[@class='prices-col']/a[1]/div[2]")
    for i in tags:
        dorms_price.append(i.text)
        
    
    time.sleep(2)
    
    #to Click on next button
    try:
        nxt_btn=driver.find_element(By.XPATH,"//div[@class='pagination-item pagination-next']") 
        nxt_btn.click()
        time.sleep(2)
    except:
        break

In [596]:
len(urls)
# distance

49

In [620]:
results={}

reviews = []
over_all = []
facilities = []
descriptions = []
rating = []

# Scraping data from each product
for url in urls:
    driver.get(url)                                                        
    print("Scraping URL = ", url)                                                       
    time.sleep(2)
    
    
    for _ in range(50):
        driver.execute_script("window.scrollBy(0,1000)")
    time.sleep(1)
    
    #Scraping rating
    try:
        tags = driver.find_element(By.XPATH,"//div[@class='reviews-and-ratings'][1]/div[1]/div/div[1]")      
        rating.append(tags.text)
    except NoSuchElementException:
        rating.append('-')
        
    #Scraping total reviews
    try:
        tags = driver.find_element(By.XPATH,"//div[@class='reviews-and-ratings'][1]/div[1]/div/div[2]/div[2]")      
        reviews.append(tags.text.replace(' Total Reviews',''))
    except NoSuchElementException:
        reviews.append('-')
        
        
    #Scraping overall reviews
    try:
        tags = driver.find_element(By.XPATH,"//div[@class='reviews-and-ratings'][1]/div[1]/div/div[2]/div[1]")      
        over_all.append(tags.text)
    except NoSuchElementException:
        over_all.append('-')
        
    #Scraping facilities
    try:
        tags = driver.find_element(By.XPATH,"//ul[@class='facility-sections']")
        facilities.append(tags.text)
    except NoSuchElementException:
        facilities.append('-')
        
    #Clicking on read more button
    try:
        read_more = driver.find_element(By.XPATH,'//a[@class="toggle-content"]')     
        read_more.click()
    except:
        print("skip show more")

    #Scraping descriptions
    try:
        tags = driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[2]/section/div[6]/div/div[2]/div/div/div[2]")
        descriptions.append(tags.text)
    except NoSuchElementException:
        descriptions.append('-')
        
    
#     break

Scraping URL =  https://www.hostelworld.com/pwa/hosteldetails.php/St-Christopher-s-Village/London/502?from=2022-10-30&to=2022-11-02&guests=2
Scraping URL =  https://www.hostelworld.com/pwa/hosteldetails.php/Prime-Backpackers-Angel/London/93902?from=2022-10-30&to=2022-11-02&guests=2
Scraping URL =  https://www.hostelworld.com/pwa/hosteldetails.php/Safestay-London-Elephant-Castle/London/55551?from=2022-10-30&to=2022-11-02&guests=2
Scraping URL =  https://www.hostelworld.com/pwa/hosteldetails.php/Astor-Museum-Inn/London/520?from=2022-10-30&to=2022-11-02&guests=2
Scraping URL =  https://www.hostelworld.com/pwa/hosteldetails.php/Hostel-One-Notting-Hill/London/291433?from=2022-10-30&to=2022-11-02&guests=2
Scraping URL =  https://www.hostelworld.com/pwa/hosteldetails.php/London-Backpackers/London/527?from=2022-10-30&to=2022-11-02&guests=2
Scraping URL =  https://www.hostelworld.com/pwa/hosteldetails.php/NX-London-Hostel/London/15204?from=2022-10-30&to=2022-11-02&guests=2
Scraping URL =  https

In [621]:
len(hostel_name),len(distance),len(pvt_prices),len(dorms_price),len(rating),len(reviews),len(over_all),len(facilities),len(descriptions)


(49, 49, 49, 49, 49, 49, 49, 49, 49)

In [622]:
results = pd.DataFrame({})
results['Hostel_Name'] = hostel_name
results['Distance fron city centre'] = distance
results['Ratings'] = rating
results['Total_reviews'] = reviews
results['Facilities'] = facilities
results['Overall Reviews'] = over_all
results['Privates from price'] = pvt_prices
results['Dorms from price'] = dorms_price
results['Description'] = descriptions

In [623]:
results

,Hostel_Name,Distance fron city centre,Ratings,Total_reviews,Facilities,Overall Reviews,Privates from price,Dorms from price,Description
0,St Christopher's Village,1.8km,8.2,11621,Free\nLinen Included Free City Maps Free WiFi ...,Fabulous,Privates From\nRs11785.94 Rs10607,Dorms From\nRs2459.77 Rs2214,COVID 19 Policy Update.\nIn response to Corona...
1,Prime Backpackers Angel,3.6km,8.4,667,Free\nLinen Included Free City Maps Towels Inc...,Fabulous,No Privates Available,Dorms From\nRs2661,The hostel is housed in a historic building in...
2,Safestay London Elephant & Castle,1.7km,7.1,4610,Free\nLinen Included Free City Maps Free WiFi\...,Very Good,No Privates Available,Dorms From\nRs2289,Safestay at Elephant & Castle is ideal if you ...
3,Astor Museum Inn,2.2km,8.2,9101,Free\nLinen Included Free City Maps Free WiFi\...,Fabulous,Privates From\nRs10459,No Dorms Available,With the British Museum literally on its doors...
4,Hostel One Notting Hill,5.5km,9.3,1548,Free\nLinen Included Free WiFi Free Internet A...,Superb,Privates From\nRs13928,Dorms From\nRs3347,The perfect place for solo travelers to connec...
5,London Backpackers,11.9km,8.1,4353,Free\nLinen Included Free City Maps Free WiFi ...,Fabulous,Privates From\nRs6308,Dorms From\nRs1917,IMPORTANT NOTICE: \n\nALL BOOKINGS THAT FALL B...
6,NX London Hostel,6.1km,7.5,1292,Free\nFree Breakfast Linen Included Free Parki...,Very Good,Privates From\nRs7283.65 Rs5827,Dorms From\nRs1754.75 Rs1404,Welcome to NX London Hostel!\n\nA Friendly bac...
7,Selina Camden,5.5km,8.0,38,Free\nLinen Included Towels Included Free WiFi...,Fabulous,Privates From\nRs22125,Dorms From\nRs7934.53 Rs6744,"Among underground music venues, innovative mus..."
8,Book a Bed Hostels,6.9km,6.2,1212,Free\nLinen Included Free City Maps Free WiFi ...,Good,Privates From\nRs7753.69 Rs6978,No Dorms Available,Welcome to Venture Hostel.Venture Hostel is a ...
9,No.8 Willesden Hostel London,10km,6.7,4875,Free\nLinen Included Free WiFi\nGeneral\nSecur...,Good,Privates From\nRs3047,No Dorms Available,At No.8 we take pride in our Customer Service ...
